In [1]:
import numpy as np
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from keras.initializers import glorot_normal
from keras import optimizers

/Users/martinfigueroapadilla/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# extract data from a csv
# notice the cool options to skip lines at the beginning
# and to only take data from certain columns
training = np.genfromtxt('data/tweets.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None)

# create our training data from the tweets
train_x = [x[1] for x in training]
# index all the sentiment labels
train_y = np.asarray([x[0] for x in training])
print(train_y)

[0 0 0 ... 2 2 2]


/Users/martinfigueroapadilla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  after removing the cwd from sys.path.


In [3]:
# only work with the 3000 most popular words found in our dataset
max_words = 5000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(train_x)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index
# Let's save this out so we can use it later
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, max(train_y) + 1)

In [4]:
'''
model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
'''

"\nmodel = Sequential()\nmodel.add(Dense(512, input_shape=(max_words,), activation='relu'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(256, activation='sigmoid'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(3, activation='softmax'))\n"

In [5]:
out_dimension = 3
nHidden = 17
nInputDimensions = train_x.shape[1]

model = Sequential()
model.add(Dense(nHidden, input_dim=nInputDimensions, activation='elu', kernel_initializer=glorot_normal(seed=0)))

for i in range(0, 7): # capas
    model.add(Dense(nHidden, activation='relu', kernel_initializer=glorot_normal(seed=0))) # agregar capa

model.add(Dense(out_dimension, activation='sigmoid', kernel_initializer=glorot_normal(seed=0)))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(train_x, train_y, batch_size=5, epochs=100, verbose=1, validation_split=0.1, shuffle=True)

Train on 8373 samples, validate on 931 samples
Epoch 1/100
8373/8373 [==============================] - 16s 2ms/step - loss: 0.4477 - acc: 0.7822 - val_loss: 0.7003 - val_acc: 0.6552
Epoch 2/100
8373/8373 [==============================] - 16s 2ms/step - loss: 0.2389 - acc: 0.8970 - val_loss: 0.8717 - val_acc: 0.6549
Epoch 3/100
8373/8373 [==============================] - 16s 2ms/step - loss: 0.1717 - acc: 0.9267 - val_loss: 1.2333 - val_acc: 0.5961
Epoch 4/100
8373/8373 [==============================] - 18s 2ms/step - loss: 0.1391 - acc: 0.9405 - val_loss: 1.7363 - val_acc: 0.5786
Epoch 5/100
8373/8373 [==============================] - 19s 2ms/step - loss: 0.1141 - acc: 0.9487 - val_loss: 1.5877 - val_acc: 0.6230
Epoch 6/100
8373/8373 [==============================] - 19s 2ms/step - loss: 0.0953 - acc: 0.9562 - val_loss: 2.5041 - val_acc: 0.5764
Epoch 7/100
8373/8373 [==============================] - 21s 3ms/step - loss: 0.0875 - acc: 0.9605 - val_loss: 2.5037 - val_acc: 0.5850
E

8373/8373 [==============================] - 16s 2ms/step - loss: 0.0391 - acc: 0.9795 - val_loss: 4.4179 - val_acc: 0.6169
Epoch 61/100
8373/8373 [==============================] - 17s 2ms/step - loss: 0.0389 - acc: 0.9803 - val_loss: 4.1424 - val_acc: 0.6244
Epoch 62/100
8373/8373 [==============================] - 17s 2ms/step - loss: 0.0405 - acc: 0.9799 - val_loss: 4.2081 - val_acc: 0.5926
Epoch 63/100
8373/8373 [==============================] - 16s 2ms/step - loss: 0.0420 - acc: 0.9797 - val_loss: 3.8012 - val_acc: 0.6319
Epoch 64/100
8373/8373 [==============================] - 18s 2ms/step - loss: 0.0388 - acc: 0.9801 - val_loss: 4.0440 - val_acc: 0.6219
Epoch 65/100
8373/8373 [==============================] - 18s 2ms/step - loss: 0.0389 - acc: 0.9801 - val_loss: 4.0247 - val_acc: 0.6348
Epoch 66/100
8373/8373 [==============================] - 21s 2ms/step - loss: 0.0390 - acc: 0.9803 - val_loss: 4.1436 - val_acc: 0.6301
Epoch 67/100
8373/8373 [==============================

In [8]:
model_json = model.to_json()
with open('model.h5', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('weights.h5')